In [1]:
import pandas as pd
from datetime import datetime
import numpy as np

In [2]:
import sys
import os
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), 'src')))

In [5]:
data = pd.read_csv(r'D:\workspace\PET\technical_phase_data.csv')
data.head()

,ID,date,merch_category,merch_postal_code,transaction_type,spendamt,nb_transactions
0,1,2019-01-01,Grocery Stores/Supermarkets,8700000,ONLINE,11238.128450,160
1,1,2019-01-01,Grocery Stores/Supermarkets,500034,ONLINE,12848.165221,183
2,2,2019-01-01,Grocery Stores/Supermarkets,110621,ONLINE,12116.165569,173
3,3,2019-01-01,Hotels/Motels,8900000,OFFLINE,7745.998879,38
4,4,2019-01-01,Restaurants,111941,OFFLINE,6927.424754,173


Public info about the column names

In [6]:
txn_channel_col = "transaction_type"
category_col = "merch_category"
time_col = "date"
postal_code_col = "merch_postal_code"
num_txns_col = "nb_transactions"

Categorising cities in the data

In [7]:
def categorize_city(code):
    if code.startswith("5"):
        return "Medellian"
    elif code.startswith("11"):
        return "Bogota"
    elif code.startswith("70"):
        return "Brasilia"
    else:
        return "Santiago"

In [8]:
age_groups = ['0-4', '5-9', '10-14', '15-19', '20-24', '25-29', '30-34', '35-39', '40-44', '45-49', '50-54', '55-59', '60-64', '65-69', '70-74', '75+']

In [9]:
consumption_distribution_raw = pd.read_csv('consumption_distribution.csv')
categories = consumption_distribution_raw['categories'].values
consumption_distribution = {}
for category in categories:
    consumption_distribution[category] = consumption_distribution_raw[consumption_distribution_raw['categories'] == category].values[0][:-1]

In [8]:
scaling_factor = pd.read_csv('fractions_offline.csv')['0'].values

In [9]:
from DP_epidemiology.contact_matrix import get_age_group_count_map
week ="2021-01-05"
start_date = datetime.strptime(week, '%Y-%m-%d')
end_date = datetime.strptime(week, '%Y-%m-%d')
cities = data[postal_code_col].astype(str).apply(categorize_city).unique()

In [10]:
counts_per_city = []
for city in cities:
    counts = get_age_group_count_map(data, age_groups, consumption_distribution, start_date, end_date, city)
    counts_per_city.append(list(counts.values()))

In [ ]:
age_groups = ['0-4', '5-9', '10-14', '15-19', '20-24', '25-29', '30-34', '35-39', '40-44', '45-49', '50-54', '55-59', '60-64', '65-69', '70-74', '75+']
consumption_distribution_raw = pd.read_csv('consumption_distribution.csv')
categories = consumption_distribution_raw['categories'].values
consumption_distribution = {}
for category in categories:
    consumption_distribution[category] = consumption_distribution_raw[consumption_distribution_raw['categories'] == category].values[0][:-1]
fraction_offline_raw = pd.read_csv('fractions_offline.csv')
fraction_offline = fraction_offline_raw['0'].values
from DP_epidemiology.contact_matrix import get_age_group_count_map
week ="2021-01-05"
start_date = datetime.strptime(week, '%Y-%m-%d')
end_date = datetime.strptime(week, '%Y-%m-%d')
cities = data["merch_postal_code"].astype(str).apply(categorize_city).unique()
counts_per_city = []
for city in cities:
    counts = get_age_group_count_map(data, age_groups, consumption_distribution, start_date, end_date, city)
    counts_per_city.append(list(counts.values()))

In [11]:
P = np.array([4136344, 4100716, 3991988, 3934088, 4090149, 4141051, 3895117, 3439202,
              3075077, 3025100, 3031855, 2683253, 2187561, 1612948, 1088448, 1394217])  
from DP_epidemiology.contact_matrix import get_contact_matrix_country
estimated_contact_matrix = get_contact_matrix_country(counts_per_city, P, scaling_factor)

In [12]:
estimated_contact_matrix

array([[0.09639499, 0.12729707, 0.23459434, 0.41401698, 0.36048598,
        0.26723388, 0.21151241, 0.1940109 , 0.18478214, 0.13886226,
        0.13810061, 0.1138968 , 0.09122256, 0.06328202, 0.04659307,
        0.02991427],
       [0.12620061, 0.16665576, 0.30711986, 0.54200709, 0.47192923,
        0.34987358, 0.27687683, 0.25394952, 0.24191389, 0.18202519,
        0.18046582, 0.14933275, 0.11932049, 0.0828854 , 0.06100833,
        0.03909869],
       [0.22640714, 0.29897676, 0.55092938, 0.97226376, 0.84656522,
        0.62773045, 0.49656496, 0.45536685, 0.43398723, 0.3275752 ,
        0.32224618, 0.26887916, 0.21358051, 0.1488641 , 0.10948895,
        0.06984912],
       [0.39377267, 0.51998324, 0.958162  , 1.69092516, 1.47231947,
        1.09178936, 0.86355278, 0.79186499, 0.75479419, 0.5702682 ,
        0.55965221, 0.46815897, 0.3712068 , 0.25899607, 0.19044626,
        0.12132627],
       [0.35646004, 0.47071313, 0.86738183, 1.53072478, 1.33282797,
        0.98832319, 0.78176311, 

In [ ]:
import matplotlib.pyplot as plt

diff = estimated_C 

plt.figure(figsize=(10, 8))
plt.imshow(diff, cmap = 'coolwarm', interpolation='none')
plt.colorbar(label = 'Diff')